# 6. Tuning

In [1]:
MODEL = 'ELR'
TUNING_MODEL = 'guanganb'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
import os
import pickle
py_file_location = "./drive/My Drive/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

py_file_location = "./drive/My Drive/LAB/BrownlowH2H/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

import NingXiang

import pandas as pd

In [4]:
ningxiang = NingXiang.NingXiang()

NingXiang Initialised


Read in and Prepare Data

In [5]:
train_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/train_data.parquet')
val_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/val_data.parquet')
test_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/test_data.parquet')

In [6]:
train_other_info = train_data[['game_id', 'player1', 'player2']]
val_other_info = val_data[['game_id', 'player1', 'player2']]
test_other_info = test_data[['game_id', 'player1', 'player2']]

In [7]:
train_x = train_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
train_y = train_data['Brownlow Votes']
val_x = val_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
val_y = val_data['Brownlow Votes']
test_x = test_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
test_y = test_data['Brownlow Votes']

In [8]:
# ningxiang.read_in_train_data(train_x, train_y)
# ningxiang.set_model_type("Regression")
# ningxiang_output = ningxiang.get_rf_based_feature_combinations(3)

# with open(f'./drive/My Drive/LAB/BrownlowH2H/models/NingXing_output.pickle', 'rb') as f:
#     ningxiang_output = pickle.load(f)

In [9]:
with open('./drive/My Drive/LAB/BrownlowH2H/models/H2H_NingXiang.pickle', 'rb') as f:
  ningxiang_output = pickle.load(f)

Set parameters

In [10]:
import GuangAnB_H2H

In [11]:
guangan = GuangAnB_H2H.GuangAnB()

GuangAn Initialised


In [12]:
from models.EnsembleLinearRegression import Ensemble_Lasso as ELR

In [13]:
guangan.read_in_model(ELR, 'Regression')

Successfully read in model <class 'models.EnsembleLinearRegression.Ensemble_Lasso'>, which is a Regression model


In [14]:
guangan.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data


In [15]:
parameter_choices = {
    'alpha': set(range(-4, 3)),
    'ensemble_n_estimators': set(range(25, 401)),
    'ensemble_max_features': (0.4, 0.85),
    'ensemble_max_samples': (0.4, 0.85),
    'ensemble_weighted': (True, False)
}

In [16]:
guangan.set_hyperparameters(parameter_choices)

Successfully recorded hyperparameter choices


In [17]:
guangan.read_in_transform({'alpha': '10^'})

Updated transform dictionary: {'alpha': '10^', 'ensemble_n_estimators': False, 'ensemble_max_features': False, 'ensemble_max_samples': False, 'ensemble_weighted': False}


In [18]:
guangan.read_in_categorical(['ensemble_weighted'])

Updated categorical dictionary: {'alpha': False, 'ensemble_n_estimators': False, 'ensemble_max_features': False, 'ensemble_max_samples': False, 'ensemble_weighted': True}
Updated original bounds dict: [({0, 1, 2, -1, -4, -3, -2}, 'alpha'), ({25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191

In [19]:
guangan.set_non_tuneable_hyperparameters({'random_state': 19260817, 'n_jobs': -1})

Successfully recorded non_tuneable_hyperparameter choices


In [20]:
guangan.set_features(ningxiang_output)

Successfully recorded tuneable feature combination choices and updated relevant internal structures


In [21]:
try:
  guangan.read_in_tuning_result_df(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv', f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')
  print('True')
except:
  print('False')

Successfully read in tuning result of 535 rows
True


In [22]:
guangan.set_tuning_result_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')

Successfully set tuning output address


In [23]:
guangan.set_best_model_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tmp_models/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')

Successfully set best model output address


In [24]:
guangan.train_other_info = train_other_info
guangan.val_other_info = val_other_info
guangan.test_other_info = test_other_info

In [25]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
guangan.tune()

Streaming output truncated to the last 5000 lines.
        Has trained 671 combinations so far
Already Trained and Tested combination {'alpha': 2, 'ensemble_n_estimators': 400, 'ensemble_max_features': 0.85, 'ensemble_max_samples': 0.625, 'ensemble_weighted': False, 'features': 0}, which had val score of 0.2117
        Current best combo: (-1, 25, 0.4, 0.4, False, 18) with val score 0.608. 
        Has trained 671 combinations so far
Already Trained and Tested combination {'alpha': 2, 'ensemble_n_estimators': 400, 'ensemble_max_features': 0.85, 'ensemble_max_samples': 0.625, 'ensemble_weighted': False, 'features': 18}, which had val score of 0.2117
        Current best combo: (-1, 25, 0.4, 0.4, False, 18) with val score 0.608. 
        Has trained 671 combinations so far
Already Trained and Tested combination {'alpha': 2, 'ensemble_n_estimators': 400, 'ensemble_max_features': 0.85, 'ensemble_max_samples': 0.85, 'ensemble_weighted': False, 'features': 0}, which had val score of 0.2117
 

In [ ]:
guangan.view_best_combo_and_score()